# Dual Simplex algorithm

## Implementation

In [1]:
import numpy as np
import math
from fractions import Fraction
import matplotlib.pyplot as plt

In [2]:
# Print the tableau matrix.
def pretty_print_tableau(tableau):
    rows = []
    for row in tableau:
        formatted_row = ["{:>6}".format(str(element)) for element in row]
        rows.append("[ " + " ".join(formatted_row) + "]")
    print("\n".join(rows))

In [3]:
# Pivot operations.
# (Minipivot operation is performed only on the row0).
def pivot(tableau, n, m, t, h, minipivot=False):
    save = tableau[t, h]
    for j in range(n + 1):
        tableau[t, j] = tableau[t, j] / save
    for i in range(m + 1):
        if i != t and tableau[i, h] != 0:
            save = tableau[i, h]
            for j in range(n + 1):
                tableau[i, j] = tableau[i, j] - save * tableau[t, j]
        if minipivot:
            break

In [4]:
# Dual Simplex algorithm.
def dual_simplex(tableau, n, m, beta, x_opt=None, verbose=False):
    unbounded = False
    optimal = False

    itr = 0 # Num of iteration.

    while not optimal and not unbounded:
        if verbose:
            print(f"Current tableau - Itr: {itr}")
            pretty_print_tableau(tableau)

        candidates = [] # Save all the basic variables < 0.
        # Optimality check.
        optimal = True
        for i in range(1, m + 1):
            if tableau[i, 0] < 0:
                optimal = False
                candidates.append(i)

        if not optimal:
            # Choose the var that leaves that basis, according to Bland's rule
            candidates = np.array(candidates)
            t = candidates[0]
            for c in candidates[1:]:
                if beta[c-1] < beta[t]:
                    t = c
                    
            if verbose:
                print(f"x[{beta[t-1]}] leaves the basis.")

            # Unbounded check.
            unbounded = True
            for j in range(1, n + 1):
                if tableau[t, j] < 0:
                    unbounded = False
                    break
            if not unbounded:
                cj_over_aj = np.zeros(n) # Stores the candidates values.
                for j in range(1, n + 1):
                    if tableau[t, j] < 0:
                        cj_over_aj[j-1] = tableau[0, j] / abs(tableau[t, j])
                    else:
                        cj_over_aj[j-1] = float('inf') # Otherwise i can't use 
                                                       # the index returned 
                                                       # by argmin in the tableau.
                                                       # (Sort of placeholder)
                
                # Choose the columns that enters the basis according to Bland's rule.
                h = np.argmin(cj_over_aj)
                h += 1 # Need this to access the corresponding cols in the tableau.

                if verbose:
                    print("Current pivot element =", tableau[t, h])

                pivot(tableau, n, m, t, h) # Update the tableau.

                if verbose:
                    print(f"x[{h}] enters the basis.")
                    print()
                beta[t-1] = h # Update the basis.
                itr += 1

    # Print the optimal solution, if necessary.
    if optimal:
        if verbose:
            print("Optimal solution:")
            print(f"x_B = {[f"x[{i}]" for i in beta]} = {[str(tableau[i, 0]) for i in range(1, m + 1)]}.")
        # Save optimal solution, if necessary.
        if x_opt is not None:
            for i, v in enumerate([tableau[i, 0] for i in range(1, m + 1)]):
                x_opt[i] = v
        minus_bar_c0 = -1 * tableau[0, 0]
        if verbose:
            print(f"Optimal value: {minus_bar_c0}")
        return minus_bar_c0

    print("The problem is unbounded.")
    return float('-inf')

## Define a linear problem

In [5]:
# Define the problem.
A = np.array(
    [[-2., -2., -1., 1., 0.],
     [-1., -2., -3., 0., 1.]], dtype=object)
b = np.array([-6., -5.], dtype=object)
c = np.array([3., 4., 5., 0., 0.], dtype=object)

to_fraction = lambda x: Fraction(x)
vfun = np.vectorize(to_fraction)
A = vfun(A)
b = vfun(b)
c = vfun(c)

# Specify the basis and the non-basic variables.
beta = [4, 5]

# Number of rows and columns
m, n = A.shape 

# Init the tableau in his original form.
# Add row0.
tableau = np.array(np.insert(c, 0, 0))
# Add b and A.
tableau = np.vstack((tableau, np.column_stack((b, A))))

In [6]:
z_opt = dual_simplex(tableau, n, m, beta, verbose=True)
print("Optimal solution is z_opt =", z_opt)

Current tableau - Itr: 0
[      0      3      4      5      0      0]
[     -6     -2     -2     -1      1      0]
[     -5     -1     -2     -3      0      1]
x[4] leaves the basis.
Current pivot element = -2
x[1] enters the basis.

Current tableau - Itr: 1
[     -9      0      1    7/2    3/2      0]
[      3      1      1    1/2   -1/2      0]
[     -2      0     -1   -5/2   -1/2      1]
x[5] leaves the basis.
Current pivot element = -1
x[2] enters the basis.

Current tableau - Itr: 2
[    -11      0      0      1      1      1]
[      1      1      0     -2     -1      1]
[      2      0      1    5/2    1/2     -1]
Optimal solution:
x_B = ['x[1]', 'x[2]'] = ['1', '2'].
Optimal value: 11
Optimal solution is z_opt = 11
